In [1]:
!pip install tensorflow==1.5
!pip install keras

    100% |████████████████████████████████| 44.4MB 18kB/s eta 0:00:011  8% |██▉                             | 4.0MB 1.5MB/s eta 0:00:27    39% |████████████▋                   | 17.6MB 2.9MB/s eta 0:00:10    90% |████████████████████████████▉   | 40.0MB 1.1MB/s eta 0:00:04
    100% |████████████████████████████████| 7.1MB 97kB/s eta 0:00:011
    100% |████████████████████████████████| 13.9MB 55kB/s eta 0:00:01 0% |▏                               | 71kB 211kB/s eta 0:01:06    36% |███████████▌                    | 5.0MB 1.0MB/s eta 0:00:09    43% |█████████████▊                  | 6.0MB 1.3MB/s eta 0:00:07    47% |███████████████▏                | 6.6MB 1.2MB/s eta 0:00:07    88% |████████████████████████████▎   | 12.2MB 1.5MB/s eta 0:00:02
    100% |████████████████████████████████| 92kB 1.5MB/s ta 0:00:01
    100% |████████████████████████████████| 3.0MB 265kB/s ta 0:00:011    58% |██████████████████▉             | 1.8MB 1.4MB/s eta 0:00:01
    100% |████████████████████████████████| 

In [2]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
import numpy as np

Using TensorFlow backend.


### data setup

#### load input and output sentance vectors

In [3]:
word_em_in = './data/docspace.tsv'

# load word embeddings
word_embeddings = {}
with open(word_em_in, 'r', encoding='utf-8') as w_in:
    for line in w_in:
        line = line.replace('\n','')
        line = line.split('\t')
        word_embeddings[line[0]] = np.array(line[1:]).astype(float)

print(word_embeddings['hello'].shape)

(100,)


In [4]:
conversation_in = './data/conversation.txt'

conversations_embedding_list_x = []
conversations_embedding_list_y = []
with open(conversation_in, 'r', encoding='utf-8') as c_in:
    for line in c_in:
        line = line.replace('\n','')
        line = line.split('\t')
        utterance_embedding_list = []
        convs_len = len(line)
        if convs_len > 1:
            for utterance in line:
                sentance_embedding = np.zeros(100,)
                wcount = 0.0
                for word in utterance.split(' '):
                    sentance_embedding = np.add(sentance_embedding, word_embeddings[word])
                    wcount = wcount + 1.0
                sentance_embedding = np.divide(sentance_embedding, wcount)
                utterance_embedding_list.append(sentance_embedding)
            conversations_embedding_list_x.append(np.array(utterance_embedding_list[0:convs_len-1]))
            conversations_embedding_list_y.append(np.array(utterance_embedding_list[1:]))

print(len(conversations_embedding_list_x))

83096


In [15]:
conversations_embedding_list_x[4].shape

(4, 100)

In [5]:
print(conversations_embedding_list_x[1])
print(conversations_embedding_list_y[1])

[[ 3.95882023e-03  4.87052308e-03 -3.83257615e-03 -4.09026069e-03
   1.46702823e-03 -2.40922400e-03 -1.52621231e-03  4.75336154e-04
   3.02166946e-03  2.86797508e-03 -5.08084938e-03 -1.49146923e-05
  -5.18533623e-03  5.25970438e-03 -8.05769585e-03  1.11989846e-03
  -6.09444015e-03  1.28499023e-03  1.56277531e-03 -3.17969208e-03
   9.44207692e-04 -2.36952077e-03  1.70005231e-04 -1.20561015e-03
  -1.29022769e-03  3.36334077e-04 -1.51560385e-03 -2.19190562e-03
   5.63522515e-03  3.19848000e-03 -1.01176892e-02 -3.77960223e-03
   1.69216869e-03  5.55467915e-03  7.07473569e-03 -3.84569231e-04
  -2.08149769e-03  3.45842208e-03 -4.44513077e-03 -1.24904154e-03
  -1.51096923e-03 -4.64181000e-04  2.47452000e-04 -1.40265846e-03
  -1.60140400e-03  1.68608038e-03 -1.59416308e-03 -6.28483841e-03
   9.17189231e-04 -4.78098615e-03  4.01535015e-03  1.63027692e-05
  -2.11582808e-03 -3.30772462e-03  4.10048846e-03 -7.49368208e-03
  -2.66846623e-03  2.74490154e-03  3.52085846e-03  5.35713077e-03
  -4.61600

### build LSTM model

In [6]:
model = Sequential()
model.add(LSTM((100), batch_input_shape=(1, None, (100)), return_sequences=True))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, None, 100)         80400     
Total params: 80,400
Trainable params: 80,400
Non-trainable params: 0
_________________________________________________________________


In [11]:
history = model.fit(conversations_embedding_list_x, conversations_embedding_list_y, epochs=5, verbose=2)

ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 array(s), but instead got the following list of 83096 arrays: [array([[ 1.68135413e-03, -5.56713325e-03,  5.68291917e-03,
         1.57534167e-03,  5.47224942e-03, -2.18916038e-03,
        -5.38892250e-03, -2.26224167e-03,  9.26792833e-04,
        -9.52086667e-0...

In [9]:
in_x = np.random.randn(200000, 5, 100)
in_y = np.random.randn(200000, 5, 100)

In [10]:
history = model.fit(in_x, in_y, epochs=5, verbose=2)

Epoch 1/5
 - 46s - loss: 1.0013 - acc: 0.0099
Epoch 2/5


KeyboardInterrupt: 